# Geos API Example

## Run this example in [Colab](https://colab.research.google.com/github/SignalOceanSdk/SignalSDK/blob/master/docs/examples/jupyter/GeoAPI/GeoAPI%20Use%20Case.ipynb) 

## Setup

Install the Signal Ocean SDK:

In [ ]:
pip install signal-ocean

Set your subscription key acquired here: https://apis.signalocean.com/profile

In [ ]:
signal_ocean_api_key = '' #replace with your subscription key

In [ ]:
from signal_ocean import Connection
from signal_ocean.geos import GeosAPI
import numpy as np
import pandas as pd
import requests
import json

In [ ]:
connection = Connection(signal_ocean_api_key)
api = GeosAPI(connection)

## Scope

In this notebook we will show how you can get all the different kinds of Geo data available.

There are three kinds of Geo data:
* Areas
* Countries
* Ports
* GeoAssets

## Areas

### Get all available areas

In [ ]:
all_areas = api.get_areas()

In [ ]:
df_areas = pd.DataFrame([a.__dict__ for a in all_areas])

In [ ]:
df_areas.head(10)

This can create a tree-like structure.

**Example** 
* West
    * Black Sea / Sea Of Marmara
    * Atlantic America
    * Baltic / North Sea
    * Mediterranean / UK Continent 
        * UK Continent
        * Mediterranean
            * West Mediterranean
            * East Mediterranean
            * Central Mediterranean

#### Get a dataframe with all area level names

We can get a dataframe with the complete area tree by gradually merging an area type with the one below it.

In [ ]:
df_areas_all = df_areas[df_areas['location_taxonomy_id'] == 7].merge(df_areas,
                                                             how = 'left',
                                                             left_on = 'area_id',
                                                             right_on = 'parent_area_id',
                                                             suffixes = ['_type7','_type6'])[['area_id_type7',
                                                                                              'area_name_type7',
                                                                                              'area_id_type6',
                                                                                              'area_name_type6']]\
                                                      .merge(df_areas,
                                                             how = 'left',
                                                             left_on = 'area_id_type6',
                                                             right_on = 'parent_area_id')[['area_id_type7',
                                                                                           'area_name_type7',
                                                                                           'area_id_type6',
                                                                                           'area_name_type6',
                                                                                           'area_id',
                                                                                           'area_name']]\
                                                      .merge(df_areas,
                                                             how = 'left',
                                                             left_on = 'area_id',
                                                             right_on = 'parent_area_id',
                                                             suffixes = ['_type5','_type4'])[['area_id_type7',
                                                                                              'area_name_type7',
                                                                                              'area_id_type6',
                                                                                              'area_name_type6',
                                                                                              'area_id_type5',
                                                                                              'area_name_type5',
                                                                                              'area_id_type4',
                                                                                              'area_name_type4']]


In [ ]:
df_areas_all.head(10)

From there we can recreate the area tree example from above, by filetring a specific type 3 area.

In [ ]:
df_areas_all[df_areas_all['area_name_type7'] == 'West']

## Countries

### Get available countries

In [ ]:
all_countries = api.get_countries()

In [ ]:
df_countries = pd.DataFrame([a.__dict__ for a in all_countries])

In [ ]:
df_countries.head(10)

## Ports

### Get availabe ports

In [ ]:
all_ports = api.get_ports()

In [ ]:
df_ports = pd.DataFrame([a.__dict__ for a in all_ports])

In [ ]:
df_ports.head(10)

## Using all of the data above

### Get all ports in a specific level 0 area

In [ ]:
area_name = 'US Gulf'

In [ ]:
df_Ports_In_Area = df_areas[  (df_areas['area_name'] == area_name ) 
                            & (df_areas['location_taxonomy_id'] == 4)].merge(df_ports,
                                                                     how = 'left',
                                                                     left_on = 'area_id',
                                                                     right_on = 'area_id_level0',
                                                                     suffixes = ['_area','_port'])[['port_id','port_name','area_id','area_name']]

In [ ]:
df_Ports_In_Area.head(10)

### Get all ports in a specific country

In [ ]:
country_name = 'United Arab Emirates'

In [ ]:
df_Ports_In_Country = df_countries[df_countries['country_name'] == country_name].merge(df_ports,
                                                                               how = 'left',
                                                                               left_on = 'country_id',
                                                                               right_on = 'country_id',
                                                                               suffixes = ['_country','_port'])[['port_id','port_name','country_id','country_name_country']]

In [ ]:
df_Ports_In_Country.head(10)

### Get all areas in a country

Some countries can span across multiple type 0 areas. We can make that connection through **ports**.

In [ ]:
country_name = 'United States'

In [ ]:
df_countries[df_countries['country_name'] == country_name].merge(df_ports,
                                                         how = 'left',
                                                         left_on = 'country_id',
                                                         right_on = 'country_id',
                                                         suffixes = ['_country',None])[['port_id','port_name','country_id','country_name_country','area_id_level0']]\
                                                  .merge(df_areas,
                                                         how = 'left',
                                                         left_on = 'area_id_level0',
                                                         right_on = 'area_id',
                                                         suffixes = (None, '_area'))[['country_id','country_name_country','area_id', 'area_name']].area_name.unique()

#### Find the top 10 Countries with the most Ports

In [ ]:
df_Ports_country_count =  df_ports.merge(df_countries,
                                         how = 'left',
                                         left_on = 'country_id',
                                         right_on = 'country_id',
                                         suffixes =['_port','_country'])[['port_id','port_name','country_id','country_name_country']] 

In [ ]:
df_Ports_country_count.country_name_country.value_counts().head(10)

## Geo Assets

### Get availabe geo assets

In [ ]:
all_geoAssets = api.get_geoAssets()

In [ ]:
df_geoAssets = pd.DataFrame([a.__dict__ for a in all_geoAssets])

In [ ]:
df_geoAssets.head(10)